Hunter S. DiCicco

with Dr. Dongwon Lee, Ph.D.

# Multinomial SVC for Categories of News

## Diagnostics to Assess Model Performance

### Setup and Imports

In [1]:
import sys
import os

os.chdir("../")
import classifier
import validator

Retreiving data...
Retreiving data...
Retreiving data...
Retreiving data...
recall: [0.69642857 0.95089286 0.03125    0.30357143 0.89285714]
precision [0.82539683 0.87295082 0.5        0.24460432 0.50632911]
f1: [0.75544794 0.91025641 0.05882353 0.27091633 0.64620355]
{1: 68, 2: 11, 3: 217, 5: 156, 7: 24}
This model got 57.49999999999999 percent correct || 644 correct out of 1120
False negatives for real data (1)
{5: 55, 2: 13}
False negatives for fake data (2)
{5: 8, 7: 3}
False negatives for opinion data (3)
{5: 147, 2: 5, 1: 12, 7: 53}
False negatives for polarized data (5)
{7: 139, 3: 4, 2: 13}
False negatives for satire data (7)
{1: 21, 3: 3}
